In [6]:
#*********************************************************************************************************************************
#                                                                                                                                *
#                                         Deterministic Finite Automaton (DFA)                                                   *
#                                                                                                                                *
#             This code defines a Deterministic Finite Automaton (DFA) and provides functionality to parse DFAs                  *
#           from JSON strings, dictionaries, and text files. It also allows testing input strings against the DFA,               *
#         printing DFA components, and generating the language accepted by the DFA. Additionally, it includes an example         *
#                                        of DFA usage and testing with input strings.                                            *
#                                                                                                                                *
#*********************************************************************************************************************************


In [2]:
import json
from collections import defaultdict

In [3]:
class DFA:
    def __init__(self, states, alphabet, transition_function, start_state, accept_states):
        """
            Initialize the DFA with the given parameters.

                - param states: A set of states.
                - param alphabet: A set of input symbols.
                - param transition_function: A dictionary representing the transition function.
                - param start_state: The start state.
                - param accept_states: A set of accept states.
        """
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.start_state = start_state
        self.accept_states = accept_states
        self.current_state = start_state
        self.history = []
        self.validate_dfa()

    def validate_dfa(self):
        """
            Validate the DFA to ensure that it meets all necessary conditions.
        """
        if self.start_state not in self.states:
            raise ValueError(f"Start state {self.start_state} is not in states")
        if not self.accept_states.issubset(self.states):
            raise ValueError("Some accept states are not in the set of states")
        for (state, symbol), next_state in self.transition_function.items():
            if state not in self.states:
                raise ValueError(f"State {state} in transition function is not in states")
            if symbol not in self.alphabet:
                raise ValueError(f"Symbol {symbol} in transition function is not in alphabet")
            if next_state not in self.states:
                raise ValueError(f"Next state {next_state} in transition function is not in states")

    def reset(self):
        """
            Reset the DFA to the start state.
        """
        self.current_state = self.start_state
        self.history = []

    def transition(self, state, symbol):
        """
            Get the next state from the transition function.

                - param state: The current state.
                - param symbol: The input symbol.
                - return: The next state.
        """
        if (state, symbol) in self.transition_function:
            next_state = self.transition_function[(state, symbol)]
            self.history.append((state, symbol, next_state))
            return next_state
        else:
            raise ValueError(f"No transition defined for state {state} on symbol {symbol}")

    def process_string(self, input_string):
        """
            Process an input string through the DFA.

                - param input_string: The input string to process.
                - return: True if the string is accepted, False otherwise.
        """
        self.reset()
        for symbol in input_string:
            if symbol not in self.alphabet:
                raise ValueError(f"Symbol {symbol} is not in the alphabet")
            self.current_state = self.transition(self.current_state, symbol)
        return self.current_state in self.accept_states

    def is_accepting(self):
        """
            Check if the current state is an accepting state.

                - return: True if the current state is an accepting state, False otherwise.
        """
        return self.current_state in self.accept_states

    def print_dfa(self):
        """
            Print the DFA components.
        """
        print("States:", self.states)
        print("Alphabet:", self.alphabet)
        print("Transition Function:")
        for (state, symbol), next_state in self.transition_function.items():
            print(f"  δ({state}, {symbol}) -> {next_state}")
        print("Start State:", self.start_state)
        print("Accept States:", self.accept_states)

    def get_history(self):
        """
            Get the history of transitions.

                - return: A list of transitions (state, symbol, next_state).
        """
        return self.history

    def get_language(self):
            """
                Generates the accepted language of the DFA. Basic version, can be optimized or extended.

                    - return: Set of strings accepted by the DFA.
            """

            def dfs(current_state, current_string, visited):                          # Depth-first search function to traverse the DFA states
                if current_state in visited:                                          # Avoid revisiting states
                    return
                visited.add(current_state)
                if current_state in self.accept_states:                               # If the current state is an accept state, add the current string to the language
                    language.add(current_string)

                for symbol in self.alphabet:                                          # Depth-first search function to traverse the DFA states
                    next_state = self.transition_function.get((current_state, symbol))
                    if next_state:
                        dfs(next_state, current_string + symbol, visited.copy())      # Recursively explore the next state

            language = set()
            dfs(self.start_state, '', set())                                          # Start DFS from the initial state with an empty string
            return language


    def check_input_language(self, input_string):
        """
            Check if the input string belongs to the language defined by the DFA.

                - param input_string: The input string to check.
                - return: True if the input string is part of the language, False otherwise.
        """
        try:
            return self.process_string(input_string)
        except ValueError:
            return False



In [7]:
class DFAParser:
    @staticmethod
    def parse_from_file(file_path):
        """
            Parse DFA definition from a file.

                - param file_path: Path to the DFA definition file.
                - return: An instance of DFA.
        """
        with open(file_path, 'r') as file:
            dfa_data = json.load(file)
        return DFAParser.parse_from_dict(dfa_data)

    @staticmethod
    def parse_from_string(dfa_string):
        """
            Parse DFA definition from a JSON string.

                - param dfa_string: JSON string containing DFA definition.
                - return: An instance of DFA.
        """
        dfa_data = json.loads(dfa_string)
        return DFAParser.parse_from_dict(dfa_data)

    @staticmethod
    def parse_from_dict(dfa_data):
        """
            Parses DFA definition from a dictionary.

                - param dfa_data: Dictionary containing DFA definition.
                - return: DFA instance.
        """
        states = set(dfa_data['states'])                                              # Extract states.
        alphabet = set(dfa_data['alphabet'])                                          # Extract alphabet.
        # Parse transition function from dictionary.
        transition_function = {(state, symbol): next_state for state, symbol, next_state in dfa_data['transition_function']}
        start_state = dfa_data['start_state']                                         # Extract start state.
        accept_states = set(dfa_data['accept_states'])                                # Extract accept states.
        return DFA(states, alphabet, transition_function, start_state, accept_states)


    @staticmethod
    def parse_from_text_file(file_path):
        """
            Parses DFA definition from a text file with specific format.

                - param file_path: Path to DFA definition text file.
                - return: DFA instance.
        """
        with open(file_path, 'r') as file:                                          # Read lines from the text file
            lines = file.readlines()
        states = set(lines[0].strip().split())                                      # Extract states
        alphabet = set(lines[1].strip().split())                                    # Extract alphabet
        start_state = lines[2].strip()                                              # Extract start state
        accept_states = set(lines[3].strip().split())                               # Extract accept states
        transition_function = {}
        for line in lines[4:]:                                                      # Parse transition function
            state, symbol, next_state = line.strip().split()
            transition_function[(state, symbol)] = next_state
        return DFA(states, alphabet, transition_function, start_state, accept_states)


In [11]:
# This function provides a summary of the code and an example of its usage.


def banner():print("""

********************************************************************************************************************
*                                                                                                                  *
*                                     Deterministic Finite Automaton (DFA)                                         *
*                                                                                                                  *
*       The DFA class represents a Deterministic Finite Automaton with methods to validate, process strings,       *
*         and check if input strings belong to the language defined by the DFA. DFAParser provides methods         *
*                       to parse DFA definitions from files, strings, or dictionaries.                             *
*                                                                                                                  *
*                                                                                                                  *
*       ## Example usage:                                                                                          *
*                       - Parsing a DFA from a JSON string:                                                        *
*                                                                                                                  *
*                       a) dfa_definition = '{"states": ["q0", "q1", "q2"], ...}'                                  *
*                                                                                                                  *
*                       b) dfa = DFAParser.parse_from_string(dfa_definition)                                       *
*                                                                                                                  *
********************************************************************************************************************
    """)


In [13]:
# This code defines a DFA using a JSON string, parses it, and then tests various input strings against the DFA.
# Finally, it prints the results of the tests and transitions.



def main():
    banner()
    # DFA definition in JSON format (as a string for this example)
    dfa_definition = '''
    {
        "states": ["q0", "q1", "q2"],
        "alphabet": ["0", "1"],
        "transition_function": [
            ["q0", "0", "q1"],
            ["q0", "1", "q0"],
            ["q1", "0", "q2"],
            ["q1", "1", "q0"],
            ["q2", "0", "q2"],
            ["q2", "1", "q2"]
        ],
        "start_state": "q0",
        "accept_states": ["q2"]
    }
    '''

    dfa = DFAParser.parse_from_string(dfa_definition)                                 # Parse the DFA from the JSON string
    dfa.print_dfa()                                                                   # Print the DFA components
    # -----------------------------------------------------------------------------------------------------------------------------

    input_string = "001"                                                              # Test the DFA with an input string
    if dfa.process_string(input_string):
        print(f"    - The string '{input_string}' is accepted by the DFA.")
    else:
        print(f"    - The string '{input_string}' is not accepted by the DFA.")

    print("\n********************************************************************************************************************")
    print("\n ## History of transitions and current state:")                              # Print the history of transitions and the current state
    for transition in dfa.get_history():
        print(f"  δ({transition[0]}, {transition[1]}) -> {transition[2]}")
    print(f"\nCurrent state: {dfa.current_state}")

    print("\n********************************************************************************************************************")
    print("\n ## Language accepted by the DFA:", dfa.get_language())                      # Print the language accepted by the DFA
    if dfa.check_input_language(input_string):                                        # Check if a string belongs to the language
        print(f"    - The string '{input_string}' is in the language of the DFA.")
    else:
        print(f"    - The string '{input_string}' is not in the language of the DFA.")

    input_string = "010101"                                                           # Test with another input string
    if dfa.process_string(input_string):
        print(f"\n    - The string '{input_string}' is accepted by the DFA.")
    else:
        print(f"\n    - The string '{input_string}' is not accepted by the DFA.")

    print("\n********************************************************************************************************************")
    print("\n ## Updated history of transitions and current state:")                      # Print the updated history of transitions and the current state
    for transition in dfa.get_history():
        print(f"  δ({transition[0]}, {transition[1]}) -> {transition[2]}")
    print(f"\nCurrent state: {dfa.current_state}")

    print("\n********************************************************************************************************************")
    invalid_input_string = "012"                                                      # Example of checking an invalid string
    try:
        if dfa.check_input_language(invalid_input_string):
            print(f"\n    - The string '{invalid_input_string}' is in the language of the DFA.")
        else:
            print(f"\n    - The string '{invalid_input_string}' is not in the language of the DFA.")
    except ValueError as e:
        print(f"\nError: {e}")

    long_input_string = "000111000"                                                   # Example of using a longer string
    if dfa.process_string(long_input_string):
        print(f"\n    - The string '{long_input_string}' is accepted by the DFA.")
    else:
        print(f"\n    - The string '{long_input_string}' is not accepted by the DFA.")

    print("\n********************************************************************************************************************")
    print("\n ## Program finished.")                                                      # End of program
    print("\n********************************************************************************************************************\n")


if __name__ == "__main__":
    main()




********************************************************************************************************************
*                                                                                                                  *
*                                     Deterministic Finite Automaton (DFA)                                         *
*                                                                                                                  *
*       The DFA class represents a Deterministic Finite Automaton with methods to validate, process strings,       *
*         and check if input strings belong to the language defined by the DFA. DFAParser provides methods         *
*                       to parse DFA definitions from files, strings, or dictionaries.                             *
*                                                                                                                  *
*                                                             